# Load DINOv2 and import Dataset

##Install Dependencies



In [ ]:
!pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-me

In [ ]:
import weaviate

In [ ]:
import torch

In [ ]:
import urllib

In [ ]:
from weaviate.embedded import EmbeddedOptions

In [ ]:
import os

In [ ]:
from PIL import Image

In [ ]:
from torchvision import transforms

In [ ]:
import pandas as pd

In [ ]:
import json


In [ ]:
import time

In [ ]:
client = weaviate.Client(
  embedded_options=EmbeddedOptions()
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 555


##Import Pytorch Model

In [ ]:
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
model.cuda()

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 131MB/s]


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

##Clone Dataset

In [ ]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git
!mkdir /content/GroceryStoreDataset/dataset/train/Vegetables/Mushroom/
!mv /content/GroceryStoreDataset/dataset/train/Vegetables/Brown-Cap-Mushroom /content/GroceryStoreDataset/dataset/train/Vegetables/Mushroom/Brown-Cap-Mushroom

Cloning into 'GroceryStoreDataset'...
remote: Enumerating objects: 6559, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 6559 (delta 45), reused 35 (delta 35), pack-reused 6293
Receiving objects: 100% (6559/6559), 116.26 MiB | 34.59 MiB/s, done.
Resolving deltas: 100% (275/275), done.
Updating files: 100% (5717/5717), done.


In [ ]:
os.listdir('/content/GroceryStoreDataset/dataset/test/')

['Fruit', 'Vegetables', 'Packages']

In [ ]:
basedir = '/content/GroceryStoreDataset/dataset/train/'

In [ ]:
def load_image_from_url(url: str) -> Image:
  try:
    return Image.open(url).convert('RGB')
  except:
    with urllib.request.urlopen(url) as f:
      return Image.open(f).convert('RGB')

In [ ]:
def make_transform() -> transforms.Compose:
  return transforms.Compose([transforms.ToTensor()])

In [ ]:
def img2vec(image: Image) -> list:
  transform = make_transform()
  rescaled_image = image.resize(((image.width //14)*14, (image.height //14)*14))
  transformed_image = transform(rescaled_image)
  batch = transformed_image.unsqueeze(0).cuda()
  with torch.inference_mode():
    return model.forward(batch).tolist()[0]

In [ ]:
data = {}
for category in os.listdir(basedir):
  for item in os.listdir(os.path.join(basedir, category)):
    data[item] = []
    for root, dirs, files in os.walk(os.path.join(basedir, category, item)):
      data[item] += [img2vec(load_image_from_url(os.path.join(root,f))) for f in files]

In [ ]:
with open('/content/data.json', 'w') as f:
  json.dump(data, f)

In [ ]:
ingredient_obj = {
    'class': 'ingredient',
    'vectorizer': 'none',
    'properties': [
        {
          'name': 'ingredient_name',
          'dataType':['text']
        },
        {
            'name': 'entry_name',
            'dataType':['text']
        }

      ]
}

In [ ]:
client.schema.create_class(ingredient_obj)

In [ ]:
data_objs = []
for item, v in data.items():
  data_objs += [({'entry_name': item+str(i), 'ingredient_name': item}, vector) for i, vector in enumerate(v)]

In [ ]:
data_objs[0][1]
vectors = [data_objs[i][1] for i in range(len(data_objs))]
data_objs = [data_objs[i][0] for i in range(len(data_objs))]

In [ ]:
class_name = 'ingredient'

In [ ]:
client.batch.configure(batch_size=100)
with client.batch as batch:
    for i, data_obj in enumerate(data_objs):
        batch.add_data_object(
            data_obj,
            class_name,
            vector=vectors[i]  # Optional: Specify an object vector
        )

In [ ]:
with open('/content/data.json', 'w') as f:
  json.dump(data, f)

In [ ]:
response = (
    client.query
    .get("ingredient", ["ingredient_name", "entry_name"])
    .with_near_vector({
        "vector": img2vec(load_image_from_url('/content/GroceryStoreDataset/sample_images/iconic/Banana_Iconic.jpg'))})
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)


print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Ingredient": [
        {
          "_additional": {
            "distance": 0.19469732
          },
          "entry_name": "Banana31",
          "ingredient_name": "Banana"
        },
        {
          "_additional": {
            "distance": 0.20740461
          },
          "entry_name": "Banana32",
          "ingredient_name": "Banana"
        }
      ]
    }
  }
}


In [ ]:
# STEP 1 - Prepare a helper function to iterate through data in batches
def get_batch_with_cursor(collection_name, batch_size, cursor=None):
    # First prepare the query to run through data
    query = (
        client.query.get(
            collection_name,         # update with your collection name
            ["ingredient_name"] # update with the required properties
        )
        .with_additional(["id"])
        .with_limit(batch_size)
    )

    # Fetch the next set of results
    if cursor is not None:
        result = query.with_after(cursor).do()
    # Fetch the first set of results
    else:
        result = query.do()
    #print(result)
    return result["data"]["Get"][collection_name]

# STEP 2 - Iterate through the data
cursor = None
while True:
    # Get the next batch of objects
    next_batch = get_batch_with_cursor("Ingredient", 100, cursor)

    # Break the loop if empty – we are done
    if len(next_batch) == 0:
        break

    # Here is your next batch of objects
    print(next_batch)

    # Move the cursor to the last returned uuid
    cursor=next_batch[-1]["_additional"]["id"]

[{'_additional': {'id': '000e0465-622f-4fb4-8bba-a29e85b433bb'}, 'ingredient_name': 'Pepper'}, {'_additional': {'id': '0011f9b7-2aab-40b4-a3fc-3ed7afb5a7bf'}, 'ingredient_name': 'Melon'}, {'_additional': {'id': '004ad411-70e5-4d41-bb30-fc6c86d2aea9'}, 'ingredient_name': 'Juice'}, {'_additional': {'id': '00848f65-61ac-47f8-a4f0-a3a451619510'}, 'ingredient_name': 'Mango'}, {'_additional': {'id': '00b8fccd-f38d-4be5-9ac9-17c0ea18f554'}, 'ingredient_name': 'Soy-Milk'}, {'_additional': {'id': '00bb0438-1cac-4936-96d8-f60994c2964e'}, 'ingredient_name': 'Juice'}, {'_additional': {'id': '00d764d4-e6f5-4166-ad21-41b81aeab830'}, 'ingredient_name': 'Pear'}, {'_additional': {'id': '00e3c399-d1b9-4e8e-830d-023bc708384f'}, 'ingredient_name': 'Milk'}, {'_additional': {'id': '0122a3d2-34e5-4b78-84f7-7467fc865414'}, 'ingredient_name': 'Pear'}, {'_additional': {'id': '0125525d-7548-4a83-abd1-90de18ef4def'}, 'ingredient_name': 'Avocado'}, {'_additional': {'id': '013a2415-187c-432e-b3f2-aa8683192779'}, 'i

##Run Test Dataset and Calculate Accuracy & Other Metrics

In [ ]:
basedir_2 = '/content/GroceryStoreDataset/dataset/test/'

In [ ]:
data_2 = {}
for category in os.listdir(basedir_2):
  for item in os.listdir(os.path.join(basedir_2, category)):
    data_2[item] = []
    for root, dirs, files in os.walk(os.path.join(basedir_2, category, item)):
      data_2[item] += [img2vec(load_image_from_url(os.path.join(root,f))) for f in files]

In [ ]:
with open('/content/data_2.json', 'w') as f:
  json.dump(data_2, f)

In [ ]:
results_data_2 = {}
for item, values in data_2.items():
  results_data_2[item] = []
  for value in values:
    response = (
      client.query
      .get("ingredient", ["ingredient_name", "entry_name"])
      .with_near_vector({
          "vector": value})
      .with_limit(2)
      .with_additional(["distance"])
      .do()
    )
    results_data_2[item] += [response['data']['Get']['Ingredient'][0]['ingredient_name']]

In [ ]:
correct = 0
sum = 0
for item, values in results_data_2.items():
  for value in values:
    if item == value:
      correct+=1
    sum+=1

accuracy = correct/sum

### Accuracy

In [ ]:
accuracy

In [ ]:
start_time = time.time()
response = (
    client.query
    .get("ingredient", ["ingredient_name", "entry_name"])
    .with_near_vector({
        "vector": img2vec(load_image_from_url('/content/Screenshot 2024-03-27 at 11.02.57 AM.png'))})
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)


print(json.dumps(response, indent=2))
print("--- %s seconds ---" % (time.time() - start_time))
display(load_image_from_url('/content/Screenshot 2024-03-27 at 11.02.57 AM.png'))

ValueError: unknown url type: '/content/Screenshot 2024-03-27 at 11.02.57\u202fAM.png'

###Confusion Matrix

In [ ]:
metrics = {}
for keys in results_data_2.keys():
  metrics[keys] = {"TP":0, "FP":0, "TN":0, "FN":0}
for item, values in results_data_2.items():
  for value in values:
    if value == item:
      metrics[value]["TP"] += 1
    else:
      metrics[value]["FP"] += 1
      metrics[item]["FN"] += 1
  metrics[item]['TN'] = correct - metrics[value]["TP"]

metrics

{'Plum': {'TP': 21, 'FP': 1, 'TN': 2132, 'FN': 1},
 'Lemon': {'TP': 16, 'FP': 1, 'TN': 2153, 'FN': 25},
 'Pineapple': {'TP': 25, 'FP': 2, 'TN': 2128, 'FN': 0},
 'Orange': {'TP': 37, 'FP': 23, 'TN': 2116, 'FN': 19},
 'Apple': {'TP': 275, 'FP': 9, 'TN': 1878, 'FN': 1},
 'Lime': {'TP': 21, 'FP': 25, 'TN': 2132, 'FN': 9},
 'Melon': {'TP': 144, 'FP': 2, 'TN': 2009, 'FN': 9},
 'Red-Grapefruit': {'TP': 16, 'FP': 22, 'TN': 2137, 'FN': 18},
 'Papaya': {'TP': 21, 'FP': 2, 'TN': 2132, 'FN': 0},
 'Passion-Fruit': {'TP': 19, 'FP': 5, 'TN': 2134, 'FN': 8},
 'Pear': {'TP': 108, 'FP': 10, 'TN': 2045, 'FN': 0},
 'Peach': {'TP': 33, 'FP': 2, 'TN': 2120, 'FN': 3},
 'Mango': {'TP': 21, 'FP': 11, 'TN': 2132, 'FN': 10},
 'Pomegranate': {'TP': 16, 'FP': 3, 'TN': 2132, 'FN': 9},
 'Kiwi': {'TP': 36, 'FP': 4, 'TN': 2117, 'FN': 9},
 'Avocado': {'TP': 37, 'FP': 3, 'TN': 2116, 'FN': 3},
 'Banana': {'TP': 44, 'FP': 0, 'TN': 2109, 'FN': 0},
 'Nectarine': {'TP': 32, 'FP': 3, 'TN': 2121, 'FN': 3},
 'Satsumas': {'TP': 

###Precision and Recall

In [ ]:
for key, item in metrics.items():
  item['recall'] = item['TP']/(item['TP']+item['FN'])
  item['precision'] = item['TP'] / (item['TP']+item['FP'])
  item['accuracy'] = (item['TP'] + item['TN']) / (item['TP'] + item['TN'] + item['FP'] + item['FN'])

metrics

{'Plum': {'TP': 21,
  'FP': 1,
  'TN': 2132,
  'FN': 1,
  'recall': 0.9545454545454546,
  'precision': 0.9545454545454546,
  'accuracy': 0.9990719257540603},
 'Lemon': {'TP': 16,
  'FP': 1,
  'TN': 2153,
  'FN': 25,
  'recall': 0.3902439024390244,
  'precision': 0.9411764705882353,
  'accuracy': 0.9881548974943052},
 'Pineapple': {'TP': 25,
  'FP': 2,
  'TN': 2128,
  'FN': 0,
  'recall': 1.0,
  'precision': 0.9259259259259259,
  'accuracy': 0.9990719257540603},
 'Orange': {'TP': 37,
  'FP': 23,
  'TN': 2116,
  'FN': 19,
  'recall': 0.6607142857142857,
  'precision': 0.6166666666666667,
  'accuracy': 0.9808656036446469},
 'Apple': {'TP': 275,
  'FP': 9,
  'TN': 1878,
  'FN': 1,
  'recall': 0.9963768115942029,
  'precision': 0.9683098591549296,
  'accuracy': 0.9953767914932964},
 'Lime': {'TP': 21,
  'FP': 25,
  'TN': 2132,
  'FN': 9,
  'recall': 0.7,
  'precision': 0.45652173913043476,
  'accuracy': 0.984453589391861},
 'Melon': {'TP': 144,
  'FP': 2,
  'TN': 2009,
  'FN': 9,
  'recall'